In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [ ]:
#메인페이지 접속
driver.get("https://www.koreabaseball.com/Default.aspx?vote=true")

#선수 조회 창 접속
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "#lnb > li:nth-child(3) > a"))
).click()

In [ ]:
#팀 선택
teamNumb = 4

WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#cphContents_cphContents_cphContents_ddlTeam > option:nth-child({teamNumb})"))
).click()

In [ ]:
#페이지 선택
page = 3
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, f"#cphContents_cphContents_cphContents_ucPager_btnNo{page}"))
).click()

In [ ]:
element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#cphContents_cphContents_cphContents_udpRecord > div.inquiry > div"))
    )

In [ ]:
print(element.text)

In [ ]:
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#cphContents_cphContents_cphContents_udpRecord > div.inquiry > div"))
)

html_str = element.get_attribute('innerHTML')

In [148]:
#크롤링
#cphContents_cphContents_cphContents_udpRecord > div.inquiry > table > tbody
table = driver.find_element(By.CSS_SELECTOR,"#cphContents_cphContents_cphContents_udpRecord > div.inquiry > table > tbody")

In [ ]:
print(table.text)

In [149]:
table_html = table.get_attribute('innerHTML')

In [ ]:
from bs4 import BeautifulSoup as bs

soup = bs(table_html, 'html.parser')
for row in soup.find_all('tr'):
    tds = row.find_all('td')
    texts = [td.get_text(strip=True) for td in tds]
    print(texts)
    break;


In [ ]:
from bs4 import BeautifulSoup as bs
col = ['등번호', '선수명', 'kbo_id', '팀', '포지션', '생년월일', '키', '몸무게', '출신']
soup = bs(table_html, 'html.parser')
data = []
for row in soup.find_all('tr'):
    row_list = []
    for i in row.find_all('td'):
        row_list.append(i.text)
    row_list.insert(2, row.find('a')['href'].split("=")[-1])
    h, w = row_list[6].split(', ')
    row_list[6] = h
    row_list.insert(7, w)
    data.append(row_list)


In [ ]:
def html_2_df(html):
    col = ['등번호', '선수명', 'kbo_id', '팀', '포지션', '생년월일', '키', '몸무게', '출신']
    soup = bs(table_html, 'html.parser')
    data = []
    for row in soup.find_all('tr'):
        row_list = []
        for i in row.find_all('td'):
            row_list.append(i.text)
        row_list.insert(2, row.find('a')['href'].split("=")[-1])
        h, w = row_list[6].split(', ')
        row_list[6] = h
        row_list.insert(7, w)
        data.append(row_list)
    df = pd.DataFrame(data, columns=col)
    return df

In [151]:
df = html_2_df(table_html)

In [152]:
print(df)

    등번호  선수명 kbo_id   팀  포지션        생년월일      키    몸무게  \
0         반즈  52528  롯데   투수  1995-10-01  189cm   91kg   
1   122  배세종  54504  롯데   투수  2001-12-01  190cm  108kg   
2   127  배인혁  53598  롯데  내야수  2004-01-29  170cm   70kg   
3    62  백두산  54507  롯데   포수  2001-05-20  180cm   95kg   
4    35  서동욱  53569  롯데   포수  2000-03-24  175cm   88kg   
5     6  손성빈  51528  롯데   포수  2002-01-14  186cm   92kg   
6    33  손호영  50150  롯데  내야수  1994-08-23  182cm   88kg   
7    59  송재영  51594  롯데   투수  2002-06-20  181cm   84kg   
8    12  신병률  68067  롯데   투수  1996-01-30  175cm   82kg   
9     3  신윤후  69508  롯데  내야수  1996-01-05  177cm   77kg   
10   23  심재민  64017  롯데   투수  1994-02-18  182cm   92kg   
11  123  엄장윤  52565  롯데   포수  2003-10-07  179cm   78kg   
12  114  오창현  55514  롯데  외야수  2002-05-21  165cm   65kg   
13   27  유강남  61102  롯데   포수  1992-07-15  182cm   88kg   
14  112  유태웅  55512  롯데  내야수  2002-03-13  175cm   77kg   
15   91  윤동희  52591  롯데  외야수  2003-09-18  187cm   85kg   
16   55  윤성빈  

In [ ]:
import pandas as pd
from io import StringIO

def txt_2_pd(text):
    lines = text.strip().split('\n')
    col = ['등번호', '선수명', '팀', '포지션', '생년월일', '키', '몸무게', '출신']
    data = []
    for line in lines:
        tmp = line.split(' ')
        tmp[5] = tmp[5].replace(',', '')
        if not tmp[0].isdigit():
            tmp.insert(0, 'none')
        while len(tmp) != len(col):
            tmp[-2] = ' '.join(tmp[-2:])
            tmp.pop()
        data.append(tmp)
    df = pd.DataFrame(data, columns=col)
    return df

df = txt_2_pd(table)
# for i in aa:
#     print(i)

In [ ]:
print(df)

In [ ]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

In [ ]:
def get_sql(sql):
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
get_sql("show tables")

In [ ]:
df.columns

In [ ]:
col = list(df.columns)

In [ ]:
print(col)

In [ ]:
cr_table_quary = '''CREATE TABLE kbo_players (
    id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
    등번호 VARCHAR(3),
    선수명 VARCHAR(10),
    팀 VARCHAR(4),
    포지션 VARCHAR(5),
    생년월일 VARCHAR(15),
    키 VARCHAR(6),
    몸무게 VARCHAR(5),
    출신 VARCHAR(30)
)'''
get_sql(cr_table_quary)

In [ ]:
sql = "INSERT IGNORE INTO kbo_players VALUE (NULL, %s, %s, %s, %s, %s, %s, %s, %s)"
for row in df.itertuples(index=False):
            cur.execute(sql, tuple(row))

In [ ]:
get_sql("show tables")

In [ ]:
get_sql("select * from kbo_players")

In [ ]:
conn.commit()

In [ ]:
tmp = driver.find_element(By.CSS_SELECTOR,"#cphContents_cphContents_cphContents_udpRecord > div.inquiry > div").text.split()[-1]

In [ ]:
#cphContents_cphContents_cphContents_udpRecord > div.inquiry > div

In [ ]:
print(tmp)

In [ ]:
tmp

In [ ]:
type(tmp)

In [157]:
import pandas as pd
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

cur.execute("show tables")
cur.fetchall()

(('kbo_player',),
 ('naver_day_stock',),
 ('st_master',),
 ('st_master2',),
 ('stock_day',))

In [158]:
w = '88kg'
w = int(w[0:-2])
print(w)

88


In [170]:
cur.execute('show tables')
cur.fetchall()

(('kbo_player',),
 ('naver_day_stock',),
 ('st_master',),
 ('st_master2',),
 ('stock_day',))

In [172]:
cur.execute('show tables')
tables = [row[0] for row in cur.fetchall()]
if 'kbo_player' in tables:
    print('t')

t


In [167]:
cur.execute("show tables")
if 'st_master' in cur.fetchall():
    print('t')